In [1]:
import tensorflow
import keras
from keras.models import Sequential # type: ignore
from keras.layers import Dense,Dropout,LSTM # type: ignore

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [3]:
base = pd.read_csv("./treino.csv")

In [4]:
base = base.dropna()
base_treinamento = base.iloc[:,1:2].values

In [5]:
normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [6]:
previsores = []
preco_real = []

for i in range(90 , 1242) :
    previsores.append(base_treinamento_normalizada[i-90 : i , 0])
    preco_real.append(base_treinamento_normalizada[i,0])
    
previsores,preco_real = np.array(previsores), np.array(preco_real)   
previsores = np.reshape(previsores,(previsores.shape[0],previsores.shape[1],1))

In [7]:
regressor = Sequential()
regressor.add(LSTM(units = 100,return_sequences= True ,input_shape =(previsores.shape[1],1)))
regressor.add(Dropout(0.3))

c:\Users\Giovanny\Deep\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
regressor.add(LSTM(units = 50,return_sequences= True ))
regressor.add(Dropout(0.3))

In [9]:
regressor.add(LSTM(units = 50,return_sequences= True ))
regressor.add(Dropout(0.3))

In [10]:
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

In [11]:
regressor.add(Dense(units=1 , activation='linear'))
regressor.add(Dropout(0.3))

In [12]:
regressor.compile(optimizer='rmsprop',loss = 'mean_squared_error', metrics = ["mean_squared_error"])

In [13]:
regressor.fit(previsores,preco_real,epochs = 100 , batch_size = 32)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 82ms/step - loss: 0.1149 - mean_squared_error: 0.1149
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 0.0878 - mean_squared_error: 0.0878
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - loss: 0.0807 - mean_squared_error: 0.0807
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - loss: 0.0881 - mean_squared_error: 0.0881
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - loss: 0.0788 - mean_squared_error: 0.0788
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - loss: 0.0865 - mean_squared_error: 0.0865
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - loss: 0.0771 - mean_squared_error: 0.0771
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - loss: 0.0795 - mean_squared_error: 0.0795
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0874 - mean_squared_error: 0.0874
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 0.0892 - mean_squared_error: 0.0892
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━

In [14]:
base_teste = pd.read_csv("./teste.csv")

In [19]:
preco_real_teste = base.iloc[:,1:2].values
base_completa = pd.concat((base["Open"],base_teste["Open"]),axis =0)
entradas = base_completa[len(base_completa) - len(base_teste) -90 :].values
entradas = entradas.reshape(-1,1)
entradas = normalizador.transform(entradas)

X_teste = []

for i in range (90,112):
    X_teste.append(entradas[i-90:i,0])
X_teste = np.array(X_teste)
X_teste = np.reshape(X_teste,(X_teste.shape[0],X_teste.shape[1],1))
previsores = regressor.predict(X_teste)
previsores = normalizador.inverse_transform(previsores)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step


In [20]:
previsores.mean()
preco_real_teste.mean()

14.041473425120772